In [ ]:
%load_ext autoreload
%autoreload 2

import os
NBROOT = os.path.realpath(os.curdir)

In [ ]:
# Preamble
import logging
logging.basicConfig(level=logging.DEBUG)

import os
from os.path import exists, join as pjoin, realpath
from pathlib import Path

from src import bbox, gdal_slope_util as S, mbt_util as M

# ! See also src/merge_dem_fr_it.sh $

##Setup
This should probably go to another file, to include with `%run Setup.ipynb`

# France

In [ ]:
%%bash

cd $download
!wget -i ign-rge-alti-5m-urls.txt
!wget --wait=20 --random-wait --no-http-keep-alive --limit-rate=300k -nc -i piemonte-alpi-dtm5-urls.txt
!wget --wait=20 --random-wait --no-http-keep-alive --limit-rate=300k -nc -i aosta-dtm0508-urls.txt

for f in RGE*29.7z; do 7z x $f; done
cd $workdir

In [ ]:
!gdalbuildvrt -a_srs EPSG:2154 -srcnodata -99999 -vrtnodata -99999 ignalps_dem.vrt \
  $download/RGEALTI_2-0_5M_ASC_LAMB93-IGN69_*/RGEALTI/1_DONNEES_LIVRAISON_*/RGEALTI_MNT_5M_ASC_LAMB93_IGN69_*/RGEALTI_FXX_*
!time gdaldem slope $=g_zstd $=g_tile ignalps_dem.vrt ignalps-lamb-slope.tif
!time gdaldem color-relief $=g_zstd $=g_tile -alpha ignalps-lamb-slope.tif data/gdaldem-slope-oslo.clr ignalps-lamb-oslo.tif

In [ ]:
ln -s $download aoste_dtm_zips
rm -f aoste_dtm_zips.txt ; touch aoste_dtm_zips.txt ; for f in DTM0508_002_000{001..914} ; do echo "/vsizip/{aoste_dtm_zips/$f.zip}/$f.ASC" >> aoste_dtm_zips.txt ; done

gdalbuildvrt -a_srs EPSG:23032 -srcnodata -9999 -vrtnodata -9999 aoste-dem.vrt -input_file_list aoste_dtm_zips.txt
time gdaldem slope $=g_zstd aoste-dem.vrt aoste-utm32n-slope.tif
time gdaldem color-relief $=g_zstd -alpha aoste-utm32n-slope.tif gdaldem-slope-oslo.clr aoste-utm32n-oslo.tif

# Switzerland

This skips the selection of tiles to download. For a more reproducible download, check [Central-Alps.ipynb](./Central-Alps.ipynb)

In [ ]:
mkdir ch_downloads
cd ch_downloads
wget --no-http-keep-alive -nc -i ../ch.swisstopo.swissalti3d-valais-w2550-e2623-s1075-n1130.csv
cat ../ch.swisstopo.swissalti3d-z9valais-w2484-e2623-s1075-n1157.csv | rev | cut -d / -f1 | rev > _vrt_input.txt
gdalbuildvrt -input_file_list _vrt_input.txt z9valais-dem.vrt
cd ..
time gdaldem slope $=g_zstd ch_downloads/z9valais-dem.vrt valais-lv95-slope.tif

In [ ]:
resolution=2.388657133911758
extent='5.6250 43.5804 7.7343 46.558'
time gdalwarp $=g_tile \
  -co bigtiff=yes -co compress=zstd -co zstd_level=1 -co sparse_ok=true \
  -dstnodata 255 \
  -t_srs EPSG:3857 -tr $=resolution -$=resolution \
  -te_srs WGS84 -te $=extent \
  fr/ignalps-lamb-slope.tif it/piemont-utm32n-slope.tif aoste/aoste-utm32n-slope.tif \
  alex/ignalex-lamb-slope.tif ch/valais-lv95-slope.tif \
  alps/slopes-Lausanne-Jouques-Sanremo-Zermatt.tif

# Make overviews, for real!

In [ ]:
%cd ../../data

/home/eoubrayrie/code/eddy-geek/slope-ign-alti


In [ ]:
S.make_overviews('alps/slopes-AlpsW.tif', reuse=True)
# time is 40 / 20 / 10 minutes for 16/15/14

Reuse ./eslo13near-z16.mbtiles
Step 1/3 completed in 0.0 seconds
Reuse alps/slopes-AlpsWz15.tif
Reuse ./eslo13near-z15.mbtiles
Step 2/3 completed in 0.0 seconds
Reuse alps/slopes-AlpsWz15.tif
Reuse ./eslo4near-z14.mbtiles
Step 3/3 completed in 0.0 seconds
cp ./eslo13near-z16.mbtiles alps/esloAlpsW.mbtiles
<<>> ./eslo13near-z16 : zoom = 16 16 ; n = 294912 ; format = png ; bounds = 5.62500000000000089,43.5804016210360174,7.73431062698364258,46.558000000000014
<< ./eslo13near-z15 : zoom = 15 15 ; n = 73728 ; format = png ; bounds = 5.62500000000000089,43.580417165137078,7.73433208465576261,46.558000000000014
>> alps/esloAlpsW : zoom = 15 16 ; n = 368640 ; format = png ; bounds = 5.62500000000000089,43.5804016210360174,7.73433208465576261,46.558000000000014
<< ./eslo4near-z14 : zoom = 14 14 ; n = 18432 ; format = png ; bounds = 5.62500000000000089,43.580417165137078,7.73428916931152433,46.558000000000014
>> alps/esloAlpsW : zoom = 14 16 ; n = 387072 ; format = png ; bounds = 5.625000000000

In [ ]:
M.mbt_merge('alps/white9.mbtiles', dest='alps/esloAlpsW.mbtiles')

<< alps/white9 : zoom = 9 9 ; n = 18 ; format = png ; bounds = 5.62499878087688998,43.5797725532024245,7.73437378087689087,46.3159946727953837
>> alps/esloAlpsW : zoom = 9 16 ; n = 387090 ; format = png ; bounds = 5.62499878087688998,43.5797725532024245,7.73437378087689087,46.558000000000014
alps/esloAlpsW Merge of the following files:
* alps/esloAlpsW : Merge of the following files:
* eslo13near-z16 : eslo13near-z16
* eslo13near-z15 : eslo13near-z15
* eslo4near-z14 : eslo4near-z14

* white9 : white9



In [ ]:
os.system('gpxsee alps/esloAlpsW.mbtiles &')

0

In [ ]:
# make_ovr(src='alps/slopes-AlpsWz14.tif', z=13)
S.slope_mbt('eslo4near', z=13, src='alps/slopes-AlpsWz13.tif')

      sed 's/nv    0   0   0   0/nv  255 255 255 255/g' \
          ~/code/eddy-geek/TIL/geo/data/gdaldem-slope-eslo4near.clr >! /tmp/gdaldem-slope-eslo4near.clr && \
      gdaldem color-relief alps/slopes-AlpsWz13.tif /tmp/gdaldem-slope-eslo4near.clr ./eslo4near-z13.mbtiles \
          -nearest_color_entry -co TILE_FORMAT=png8 
0...10...20...30...40...50...60...70...80...90...100 - done.


'./eslo4near-z13.mbtiles'

In [ ]:
S.slope_mbt('eslo4uv', z=13, src='alps/slopes-AlpsWz13.tif')

      sed 's/nv    0   0   0   0/nv  255 255 255 255/g' \
          ~/code/eddy-geek/TIL/geo/data/gdaldem-slope-eslo4uv.clr >! /tmp/gdaldem-slope-eslo4uv.clr && \
      gdaldem color-relief alps/slopes-AlpsWz13.tif /tmp/gdaldem-slope-eslo4uv.clr ./eslo4uv-z13.mbtiles \
          -nearest_color_entry -co TILE_FORMAT=png8 
0...10...20...30...40...50...60...70...80...90...100 - done.


'./eslo4uv-z13.mbtiles'

In [ ]:
S.make_ovr(src='alps/slopes-AlpsWz14.tif', z=12)
S.slope_mbt('eslo4near', z=12, src='alps/slopes-AlpsWz13.tif')


In [ ]:
M.mbt_merge('eslo4near-z13.mbtiles', dest='alps/esloAlpsW.mbtiles')

<< eslo4near-z13 : zoom = 13 13 ; n = 4608 ; format = png ; bounds = 5.62500000000000089,43.580479341501217,7.73437500000000089,46.558000000000014
>> alps/esloAlpsW : zoom = 9 16 ; n = 391698 ; format = png ; bounds = 5.62499878087688998,43.5797725532024245,7.73437500000000089,46.558000000000014
alps/esloAlpsW Merge of the following files:
* alps/esloAlpsW : Merge of the following files:
* alps/esloAlpsW : Merge of the following files:
* eslo13near-z16 : eslo13near-z16
* eslo13near-z15 : eslo13near-z15
* eslo4near-z14 : eslo4near-z14

* white9 : white9

* eslo4near-z13 : eslo4near-z13

